In [27]:
import sqlite3
import subprocess
import re

In [28]:
def connect_db(db_path='db_contents.db'):
    try:
        conn = sqlite3.connect(db_path)
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        return None

In [29]:
# Query the database to find the appropriate command
def get_command(conn, user_input):
    query = '''
    SELECT command FROM "CMD-LIST" WHERE natural_language = ?;
    '''
    cursor = conn.execute(query, (user_input,))
    result = cursor.fetchone()
    return result[0] if result else None


In [ ]:
# Replace placeholders with user input
def replace_placeholders(command):
    # Detect placeholders like <filename>, <source>, <destination>, etc.
    placeholders = re.findall(r'<(.*?)>', command)
    
    if placeholders:
        for placeholder in placeholders:
            # Ask the user for each missing parameter
            value = input(f"Please provide a value for {placeholder}: ")
            # Replace the placeholder with the user-provided value
            command = command.replace(f"<{placeholder}>", value)
    
    return command

In [30]:
# Execute the command in the terminal
def execute_command(command):
    try:
        result = subprocess.run(command,shell=True,
                                check=True, 
                                text=True, 
                                stdout=subprocess.PIPE, 
                                stderr=subprocess.PIPE)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error executing command: {e.stderr}"

In [31]:
conn = connect_db()
if conn is None:
    print("Failed to connect to the database. Exiting.")
    exit(1)

# User input
user_input = input("Enter your command: ").strip().lower()

# Retrieve the Unix command and description based on user input
result = get_command(conn, user_input)

if result:
    command, description = result
    print(f"Executing: {description}")
    
    # Replace any placeholders like <filename>, <address>, etc.
    command_with_values = replace_placeholders(command)
    
    # Execute the command and show output
    output = execute_command(command_with_values)
    print(output)
else:
    print("Command not found in the database.")

Executing: df -h
Filesystem      Size  Used Avail Use% Mounted on
tmpfs           3.1G  2.1M  3.1G   1% /run
efivarfs        384K   95K  285K  25% /sys/firmware/efi/efivars
/dev/nvme0n1p2  938G   35G  856G   4% /
tmpfs            16G  140M   16G   1% /dev/shm
tmpfs           5.0M  8.0K  5.0M   1% /run/lock
/dev/nvme0n1p1  511M  6.2M  505M   2% /boot/efi
tmpfs           3.1G  224K  3.1G   1% /run/user/1000

